# Machine Learning on MCU - Ex3.3

This exercise will introduce you to Keras, one of the most popular frameworks used in Deep Learning.

You will train a simple multi-layer perceptron to predict Human Activity from Smartphone Accelerometer and Gyroscope Data (the same dataset you used in exercise 2).


In [2]:
# Import the necessary packages


# Note: use Tensor Flow backend for Keras as suggested on keras.io:
# "At this time, we recommend that Keras users who use multi-backend Keras with the TensorFlow backend switch to tf.keras in TensorFlow 2.0. tf.keras is better maintained and has better integration with TensorFlow features (eager execution, distribution support and other)."
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import layers, models, regularizers


from __future__ import division
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import random
random.seed(7)

In [3]:
# Load the accelerometer and gyroscope data.
with open('./HAPT Data Set/features.txt') as f:
    features = f.read().split()

print('There are {} features.'.format(len(features)))
    
with open('./HAPT Data Set/activity_labels.txt') as f:
    activity_labels = f.readlines()

activity_df = [x.split() for x in activity_labels]
print('There are {} activities.'.format(len(activity_df)))
pd.DataFrame(activity_df, columns = ['Activity_id', 'Activity_label'])


There are 561 features.
There are 12 activities.


,Activity_id,Activity_label
0,1,WALKING
1,2,WALKING_UPSTAIRS
2,3,WALKING_DOWNSTAIRS
3,4,SITTING
4,5,STANDING
5,6,LAYING
6,7,STAND_TO_SIT
7,8,SIT_TO_STAND
8,9,SIT_TO_LIE
9,10,LIE_TO_SIT


In [4]:
# Split the dataset into training, validation, and testing sets. (You can take example from ex 1 and 2.)
X_train = pd.read_table('./HAPT Data Set/Train/X_train.txt',
             header = None, sep = " ", names = list(dict.fromkeys(features)))
y_train = pd.read_table('./HAPT Data Set/Train/y_train.txt',
             header = None, sep = " ", names = ['Activity_id'])

X_test = pd.read_table('./HAPT Data Set/Test/X_test.txt',
             header = None, sep = " ", names = list(dict.fromkeys(features)))
y_test = pd.read_table('./HAPT Data Set/Test/y_test.txt',
             header = None, sep = " ", names = ['Activity_id'])

# Note: Keras takes the classes from 0, i.e. if you have classes [1, 2, 3] (as in this case), it will give you an error.
# To avoid the error, shift the class labels by -1, i.e. from [1, 2, 3] to [0, 1, 2]
#activity_df[["Activity_id"]] = activity_df[["Activity_id"]].apply(pd.to_numeric) -1

y_train = y_train -1
y_test = y_test - 1             


In [39]:
# Define batchsize and number of epochs
#inputShape = (X_train.shape[1], -1)
inputShape = X_train.shape[1]
outputShape = len(activity_df)
batchSize = 32
epochs = 10

# Declare the sequential model and design your multi-layer perceptron
model = keras.Sequential(
    [
        layers.Dense(inputShape, activation = "relu"),
        layers.Dense(64, activation = "relu"),
        layers.Dense(32, activation = "relu"),
        layers.Dense(12, activation = "softmax")
    ]
)

# Compile your model
model.build(input_shape=(1, inputShape))
model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=keras.optimizers.Adam(),
    metrics=['accuracy']
    )
# Train your model

model.fit(X_train, y_train, batch_size= batchSize, epochs=epochs)

Epoch 1/10
243/243 [==============================] - 2s 5ms/step - loss: 0.5589 - accuracy: 0.8064
Epoch 2/10
243/243 [==============================] - 1s 4ms/step - loss: 0.2147 - accuracy: 0.9208
Epoch 3/10
243/243 [==============================] - 1s 5ms/step - loss: 0.1661 - accuracy: 0.9351
Epoch 4/10
243/243 [==============================] - 1s 4ms/step - loss: 0.1228 - accuracy: 0.9533
Epoch 5/10
243/243 [==============================] - 1s 4ms/step - loss: 0.1206 - accuracy: 0.9524
Epoch 6/10
243/243 [==============================] - 1s 5ms/step - loss: 0.1020 - accuracy: 0.9609
Epoch 7/10
243/243 [==============================] - 1s 5ms/step - loss: 0.1082 - accuracy: 0.9587
Epoch 8/10
243/243 [==============================] - 1s 5ms/step - loss: 0.0868 - accuracy: 0.9659
Epoch 9/10
243/243 [==============================] - 1s 5ms/step - loss: 0.0942 - accuracy: 0.9645
Epoch 10/10
243/243 [==============================] - 1s 5ms/step - loss: 0.0907 - accuracy: 0.9681

In [28]:
# You can use model.summary() to get an overview of your model
model.summary()



Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_17 (Dense)            (1, 533)                  284622    
                                                                 
 dense_18 (Dense)            (1, 64)                   34176     
                                                                 
 dense_19 (Dense)            (1, 32)                   2080      
                                                                 
 dense_20 (Dense)            (1, 12)                   396       
                                                                 
Total params: 321,274
Trainable params: 321,274
Non-trainable params: 0
_________________________________________________________________


In [43]:
# Evaluate your model

score = model.evaluate(X_train, y_train)
print('Train accuracy:', score[1])

243/243 [==============================] - 1s 3ms/step - loss: 0.0894 - accuracy: 0.9647
Train accuracy: 0.9647225737571716


In [44]:
# Save your model once you are satisfied with it

model.save('ex3_model')

INFO:tensorflow:Assets written to: ex3_model\assets


In [45]:
# Load your saved model and test on the test data.
model = keras.models.load_model('ex3_model')
score = model.evaluate(X_test, y_test)

print('Test accuracy:', score[1])

99/99 [==============================] - 0s 3ms/step - loss: 0.2701 - accuracy: 0.9051
Test accuracy: 0.9051233530044556
